# simulation for MCS erun 201908 (in ICS)

## Description:

**Summary:**

This notebook contains a script to run a closed loop simulation with MCS, pinhole mask, and opDB. 

**Requirements:**

- Python 3
- numpy
- psycopg2
- The latest version of the PFS datamodel
    - see https://github.com/Subaru-PFS/datamodel

**Note:**

opDB is not a requirement, but the schema diagram is useful (see https://github.com/Subaru-PFS/spt_operational_database/tree/tickets/SURVEY-13)

## load basic modules

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys, os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import MultipleLocator
from matplotlib.collections import PolyCollection
import mpl_toolkits.axes_grid1
import scipy as sp
import numpy as np
import datetime

In [2]:
import opdb
import pfs_sim_erun as erunsim

/Users/kiyoyabe/tmp/PyFITS-3.5/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


## simple closed-loop simulation

### preparation

#### database connection

In [3]:
hostname = 'hostname'      # <-- edit here
port = '5432'              # <-- edit here
dbname = 'mcs_erun_201908'
username = 'pfs'
passwd = 'password'        # <-- edit here

In [4]:
db = opdb.OpDB(hostname=hostname, port=port, dbname=dbname, username=username, passwd=passwd)
db.connect()

connected mcs_erun_201908


In [5]:
fiberPositions = db.get_all_fiber_position()
cobraPositions = db.get_all_cobra_position()
fiducialFiberPositions = db.get_all_fiducial_fiber_position()

### initial setup

In [6]:
db.reset()

resetting ...
... done


0

#### insert info into `Proposal`, `Program`, `Tile`

In [7]:
proposalId = 'o99999'

programId = 1
programName = 'MCS erun 201908'
programDescription = 'MCS engineering run in August 2019'
filler = False

tile = 1
tileId = programId * 10000 + tile
raCenter = 0.0
decCenter = 0.0
pa = 0.0
finished = False

In [8]:
db.insert_proposal(proposalId=proposalId)

insert into Proposal


In [9]:
db.insert_program(programId=programId, name=programName, description=programDescription, proposalId=proposalId, filler=filler)

insert into Program


In [10]:
db.insert_tile(tileId=tileId, programId=programId, tile=tile, 
               raCenter=raCenter, decCenter=decCenter, pa=pa, 
               finished=finished)

insert into Tile


### Target selection

#### preparation

##### get fiber positions

In [11]:
fiberPositions = db.get_all_fiber_position()
cobraPositions = db.get_all_cobra_position()
fiducialFiberPositions = db.get_all_fiducial_fiber_position()

nFiber = len(fiberPositions['fiberId'])
nCobra = len(cobraPositions['cobraId'])
nFiducialFiber = len(fiducialFiberPositions['ffId'])

print('# of fibers          : %4d' % (nFiber))
print('# of cobras          : %4d' % (nCobra))
print('# of fiducial fibers : %4d' % (nFiducialFiber))

# of fibers          : 1703
# of cobras          : 1574
# of fiducial fibers :  129


In [12]:
fiberIds = fiberPositions['fiberId']
fp_ftype = fiberPositions['ftype']
fp_x = fiberPositions['x']
fp_y = fiberPositions['y']

##### use fiber positions as target positions this time

In [13]:
np.random.seed(0)

msk_cobra = fp_ftype=='cobra'
msk_ff = fp_ftype=='ff'

objIds = np.arange(nFiber) + 1
ras = raCenter + np.random.uniform(-0.5, +0.5, nFiber)
decs = raCenter + np.random.uniform(-0.5, +0.5, nFiber)
tracts = np.array([12345 for i in range(nFiber)], dtype='i4')
patches = np.array(['0,0' for i in range(nFiber)], dtype='U5')
priorities = np.array([1.0 for i in range(nFiber)], dtype='f4')
targetTypeIds = np.empty(nFiber, dtype='i4')
targetTypeIds[msk_cobra] = 1
targetTypeIds[msk_ff] = 2
catIds = np.array([1 for i in range(nFiber)], dtype='i4')
catObjIds = objIds.copy()
fiberMags_g = np.array([99.0 for i in range(nFiber)], dtype='f4')
fiberMags_r = np.array([99.0 for i in range(nFiber)], dtype='f4')
fiberMags_i = np.array([99.0 for i in range(nFiber)], dtype='f4')
fiberMags_z = np.array([99.0 for i in range(nFiber)], dtype='f4')
fiberMags_y = np.array([99.0 for i in range(nFiber)], dtype='f4')
fiberMags_j = np.array([99.0 for i in range(nFiber)], dtype='f4')
fiducialExptimes = np.array([900. for i in range(nFiber)], dtype='f4')
photzs = np.array([0.0 for i in range(nFiber)], dtype='f4')
mediumResolutions = np.array([False for i in range(nFiber)], dtype='bool')
QATypeIds = np.array([1 for i in range(nFiber)], dtype='i4')
QALambdaMins = np.array([380. for i in range(nFiber)], dtype='f4')
QALambdaMaxs = np.array([1260. for i in range(nFiber)], dtype='f4')
QAThresholds = np.array([5.0 for i in range(nFiber)], dtype='f4')
QALineFluxes = np.array([0.0 for i in range(nFiber)], dtype='f4')
completenesses = np.array([0.0 for i in range(nFiber)], dtype='f4')
finished = np.array([False for i in range(nFiber)], dtype='bool')
#plt.scatter(ras, decs, marker='o', s=10, fc='red', ec='k')

#### insert info into `Target`

In [14]:
db.insert_target(programId=programId, objId=objIds, ra=ras, dec=decs, tract=tracts, patch=patches, 
                 priority=priorities, targetTypeId=targetTypeIds, catId=catIds, catObjId=catObjIds, 
                 fiberMag_g=fiberMags_g, fiberMag_r=fiberMags_r, fiberMag_i=fiberMags_i, fiberMag_z=fiberMags_z, 
                 fiberMag_y=fiberMags_y, fiberMag_j=fiberMags_j, fiducialExptime=fiducialExptimes, photz=photzs, 
                 mediumResolution=mediumResolutions, QATypeId=QATypeIds, QALambdaMin=QALambdaMins, QALambdaMax=QALambdaMaxs, 
                 QAThreshold=QAThresholds, QALineFlux=QALineFluxes, completeness=completenesses, finished=finished)

insert into Target


In [15]:
targetIds = db.get_targetId(programId=programId, objIds=objIds)

### make `pfsDesign` and `pfsDesignFiber`

#### preparation (make pfsDesign and pfsDesignId)

In [16]:
pfiNominal_x = fp_x.copy()
pfiNominal_y = fp_y.copy()

pfsDesign, pfsDesignId = erunsim.make_pfsDesign(raCenter, decCenter, fiberIds, tracts, patches, ras, decs, 
                                                catIds, objIds, targetTypeIds, fiberMags_g, 
                                                pfiNominal_x, pfiNominal_y)

print(pfsDesignId)

1277202677728382843


#### insert info into `pfsDesign`

In [17]:
db.insert_pfsDesign(pfsDesignId=pfsDesignId, tileId=tileId, 
                    raCenter=raCenter, decCenter=decCenter, paConfig=pa,
                    numSciDesigned=nFiber, numCalDesigned=0, numSkyDesigned=0, numGuideStars=0,
                    exptime=900., minExptime=900., etsVersion='1.0', etsAssigner='naive', etsExectime=datetime.datetime.now(),
                    obsolete=False)

insert into pfsDesign


#### insert info into `pfsDesignFiber`

In [18]:
onSources = np.array([True for i in range(nFiber)], dtype='bool')
etsPriorities = np.array([1 for i in range(nFiber)], dtype='i4')
etsCostFunctions = np.array(['none' for i in range(nFiber)], dtype='U10')
etsCobraMovements = np.array(['none' for i in range(nFiber)], dtype='U10')

db.insert_pfsDesignFiber(pfsDesignId=pfsDesignId, fiberId=fiberIds, targetId=targetIds, tract=tracts, patch=patches, ra=ras, dec=decs,
                         catId=catIds, objId=objIds, targetTypeId=targetTypeIds, fiberMag_g=fiberMags_g, fiberMag_r=fiberMags_r, 
                         fiberMag_i=fiberMags_i, fiberMag_z=fiberMags_z, fiberMag_y=fiberMags_y, fiberMag_j=fiberMags_j,
                         etsPriority=etsPriorities, etsCostFunction=etsCostFunctions, etsCobraMovement=etsCobraMovements, 
                         pfiNominal_x=pfiNominal_x, pfiNominal_y=pfiNominal_y, onSource=onSources)

insert into pfsDesignFiber


### make (initial) `pfsConfig` and `pfsConfigFiber`

#### preparation (define visit number)

In [19]:
visit = 1234
telEl = 90.
insRot = 0.0

#### insert info into `Visit`

In [20]:
db.insert_visit(visit=visit, visitTypeId=1, description='mcs erun 201908')

insert into Visit


#### insert info into `pfsConfig`

In [21]:
db.insert_pfsConfig(pfsConfigId=None, pfsDesignId=pfsDesignId, visit0=visit, 
                    raCenter=raCenter, decCenter=decCenter, paConfig=pa, telEl=telEl, insRot=insRot,
                    numSciAllocated=nFiber, numCalAllocated=0, numSkyAllocated=0, numGuideStars=0,
                    exptime=900., minExptime=900., allocNumIter=0, allocElapsetime=0.0, allocRmsScatter=0.0, 
                    allocExectime=datetime.datetime.now(), observed=False)

insert into pfsConfig


In [22]:
pfsConfigId = db.get_pfsConfigId(pfsDesignId=pfsDesignId, visit0=visit)
print(pfsConfigId)

1


#### make pfsConfigFiberId

In [23]:
pfsConfigFiberIds = (pfsConfigId << 12) + fiberIds
print(pfsConfigFiberIds)
print(fiberIds)

[4097 4098 4099 ... 5797 5798 5799]
[   1    2    3 ... 1701 1702 1703]


#### insert info into `pfsConfigFiber`

In [24]:
onSources = np.array([True for i in range(nFiber)], dtype='bool')
pfiCenter_x = np.zeros(nFiber)
pfiCenter_y = np.zeros(nFiber)
pfiDiff_x = np.zeros(nFiber)
pfiDiff_y = np.zeros(nFiber)
mcsCenter_x = np.zeros(nFiber)
mcsCenter_y = np.zeros(nFiber)
motorMapSummary = np.array(['none' for i in range(nFiber)], dtype='U20')
configTime = np.zeros(nFiber)

db.insert_pfsConfigFiber(pfsConfigFiberId=pfsConfigFiberIds, pfsConfigId=pfsConfigId, fiberId=fiberIds, targetId=targetIds, 
                         tract=tracts, patch=patches, ra=ras, dec=decs, catId=catIds, objId=objIds, targetTypeId=targetTypeIds, 
                         fiberMag_g=fiberMags_g, fiberMag_r=fiberMags_r, fiberMag_i=fiberMags_i, fiberMag_z=fiberMags_z, 
                         fiberMag_y=fiberMags_y, fiberMag_j=fiberMags_j,
                         pfiNominal_x=pfiNominal_x, pfiNominal_y=pfiNominal_y, pfiCenter_x=pfiCenter_x, pfiCenter_y=pfiCenter_y, 
                         pfiDiff_x=pfiDiff_x, pfiDiff_y=pfiDiff_y, mcsCenter_x=mcsCenter_x, mcsCenter_y=mcsCenter_y,
                         motorMapSummary=motorMapSummary, configTime=configTime, onSource=onSources)


insert into pfsConfigFiber


### 0th configuration

#### take MCS exposure

In [25]:
## skipped ##

#### get MCS results

In [26]:
## results ##
np.random.seed(0)

datatime = datetime.datetime.now()
frameId = 1
moveId = 1
fiberId_mcsData = fiberIds.copy()
centroidx = np.zeros(nFiber) + np.random.uniform(0, 2048, nFiber)
centroidy = np.zeros(nFiber) + np.random.uniform(0, 2048, nFiber)
fwhmx = np.zeros(nFiber) + 3.0 + np.random.uniform(-0.5, +0.5, nFiber)
fwhmy = np.zeros(nFiber) + 3.0 + np.random.uniform(-0.5, +0.5, nFiber)
bgvalue = np.zeros(nFiber) + 1000. + np.random.uniform(-200., +200., nFiber)
peakvalue = np.zeros(nFiber) + 2000. + np.random.uniform(-200., +200., nFiber)

#### insert info into `mcsData`

In [27]:
db.insert_mcsData(datatime=datatime, frameId=frameId, moveId=moveId, fiberId=fiberId_mcsData, 
                  centroidx=centroidx, centroidy=centroidy, fwhmx=fwhmx, fwhmy=fwhmy, 
                  bgvalue=bgvalue, peakvalue=peakvalue)

insert into mcsData


#### insert info into `CobraConfig`

In [28]:
np.random.seed(0)

mcsIds, mcsCenter_x, mcsCenter_y = db.get_mcsCenter(frameId, fiberIds)

## PFI info ##
iteration = 0
exectime = datetime.datetime.now()
pfiCenter_x = pfiNominal_x + np.random.uniform(-1.0, +1.0, nFiber)
pfiCenter_y = pfiNominal_y + np.random.uniform(-1.0, +1.0, nFiber)
pfiDiff_x = pfiCenter_x - pfiNominal_x
pfiDiff_y = pfiCenter_y - pfiNominal_y
motorNumStepTheta = np.random.randint(100, 1000, nFiber)
motorNumStepPhi = np.random.randint(100, 1000, nFiber)

#plt.scatter(pfiDiff_x, pfiDiff_y, marker='o', s=10, fc='red', ec='k')

exectime_start = exectime

In [29]:
db.insert_CobraConfig(pfsConfigFiberId=pfsConfigFiberIds, pfsConfigId=pfsConfigId, fiberId=fiberIds, iteration=iteration,
                      motorNumStepTheta=motorNumStepTheta, motorNumStepPhi=motorNumStepPhi, mcsId=mcsIds,
                      pfiNominal_x=pfiNominal_x, pfiNominal_y=pfiNominal_y, pfiCenter_x=pfiCenter_x, pfiCenter_y=pfiCenter_y,
                      pfiDiff_x=pfiDiff_x, pfiDiff_y=pfiDiff_y, mcsCenter_x=mcsCenter_x, mcsCenter_y=mcsCenter_y,
                      exectime=exectime
                     )

insert into CobraConfig


### 1st configuration

#### move cobra

In [30]:
## skipped

#### take MCS exposure

In [31]:
## skipped

#### get MCS results

In [32]:
## results ##
np.random.seed(1)

datatime = datetime.datetime.now()
frameId = 2
moveId = 1
fiberId_mcsData = fiberIds.copy()
centroidx = np.zeros(nFiber) + np.random.uniform(0, 2048, nFiber)
centroidy = np.zeros(nFiber) + np.random.uniform(0, 2048, nFiber)
fwhmx = np.zeros(nFiber) + 3.0 + np.random.uniform(-0.5, +0.5, nFiber)
fwhmy = np.zeros(nFiber) + 3.0 + np.random.uniform(-0.5, +0.5, nFiber)
bgvalue = np.zeros(nFiber) + 1000. + np.random.uniform(-200., +200., nFiber)
peakvalue = np.zeros(nFiber) + 2000. + np.random.uniform(-200., +200., nFiber)

#### insert info into `mcsData`

In [33]:
db.insert_mcsData(datatime=datatime, frameId=frameId, moveId=moveId, fiberId=fiberId_mcsData, 
                  centroidx=centroidx, centroidy=centroidy, fwhmx=fwhmx, fwhmy=fwhmy, 
                  bgvalue=bgvalue, peakvalue=peakvalue)

insert into mcsData


#### insert info into `CobraConfig`

In [34]:
np.random.seed(1)

mcsIds, mcsCenter_x, mcsCenter_y = db.get_mcsCenter(frameId, fiberIds)

## PFI info ##
iteration = 1
exectime = datetime.datetime.now()
pfiCenter_x = pfiNominal_x + np.random.uniform(-1.0, +1.0, nFiber)
pfiCenter_y = pfiNominal_y + np.random.uniform(-1.0, +1.0, nFiber)
pfiDiff_x = pfiCenter_x - pfiNominal_x
pfiDiff_y = pfiCenter_y - pfiNominal_y
motorNumStepTheta = np.random.randint(100, 1000, nFiber)
motorNumStepPhi = np.random.randint(100, 1000, nFiber)

#plt.scatter(pfiDiff_x, pfiDiff_y, marker='o', s=10, fc='red', ec='k')

In [35]:
db.insert_CobraConfig(pfsConfigFiberId=pfsConfigFiberIds, pfsConfigId=pfsConfigId, fiberId=fiberIds, iteration=iteration,
                      motorNumStepTheta=motorNumStepTheta, motorNumStepPhi=motorNumStepPhi, mcsId=mcsIds,
                      pfiNominal_x=pfiNominal_x, pfiNominal_y=pfiNominal_y, pfiCenter_x=pfiCenter_x, pfiCenter_y=pfiCenter_y,
                      pfiDiff_x=pfiDiff_x, pfiDiff_y=pfiDiff_y, mcsCenter_x=mcsCenter_x, mcsCenter_y=mcsCenter_y,
                      exectime=exectime
                     )

insert into CobraConfig


### 2nd configuration

#### move cobra

In [36]:
## skipped

#### take MCS exposure

In [37]:
## skipped

#### get MCS results

In [38]:
## results ##
np.random.seed(2)

datatime = datetime.datetime.now()
frameId = 3
moveId = 1
fiberId_mcsData = fiberIds.copy()
centroidx = np.zeros(nFiber) + np.random.uniform(0, 2048, nFiber)
centroidy = np.zeros(nFiber) + np.random.uniform(0, 2048, nFiber)
fwhmx = np.zeros(nFiber) + 3.0 + np.random.uniform(-0.5, +0.5, nFiber)
fwhmy = np.zeros(nFiber) + 3.0 + np.random.uniform(-0.5, +0.5, nFiber)
bgvalue = np.zeros(nFiber) + 1000. + np.random.uniform(-200., +200., nFiber)
peakvalue = np.zeros(nFiber) + 2000. + np.random.uniform(-200., +200., nFiber)

#### insert info into `mcsData`

In [39]:
db.insert_mcsData(datatime=datatime, frameId=frameId, moveId=moveId, fiberId=fiberId_mcsData, 
                  centroidx=centroidx, centroidy=centroidy, fwhmx=fwhmx, fwhmy=fwhmy, 
                  bgvalue=bgvalue, peakvalue=peakvalue)

insert into mcsData


#### insert info into `CobraConfig`

In [40]:
np.random.seed(2)

mcsIds, mcsCenter_x, mcsCenter_y = db.get_mcsCenter(frameId, fiberIds)

## PFI info ##
iteration = 2
exectime = datetime.datetime.now()
pfiCenter_x = pfiNominal_x + np.random.uniform(-1.0, +1.0, nFiber)
pfiCenter_y = pfiNominal_y + np.random.uniform(-1.0, +1.0, nFiber)
pfiDiff_x = pfiCenter_x - pfiNominal_x
pfiDiff_y = pfiCenter_y - pfiNominal_y
motorNumStepTheta = np.random.randint(100, 1000, nFiber)
motorNumStepPhi = np.random.randint(100, 1000, nFiber)

#plt.scatter(pfiDiff_x, pfiDiff_y, marker='o', s=10, fc='red', ec='k')

In [41]:
db.insert_CobraConfig(pfsConfigFiberId=pfsConfigFiberIds, pfsConfigId=pfsConfigId, fiberId=fiberIds, iteration=iteration,
                      motorNumStepTheta=motorNumStepTheta, motorNumStepPhi=motorNumStepPhi, mcsId=mcsIds,
                      pfiNominal_x=pfiNominal_x, pfiNominal_y=pfiNominal_y, pfiCenter_x=pfiCenter_x, pfiCenter_y=pfiCenter_y,
                      pfiDiff_x=pfiDiff_x, pfiDiff_y=pfiDiff_y, mcsCenter_x=mcsCenter_x, mcsCenter_y=mcsCenter_y,
                      exectime=exectime
                     )

insert into CobraConfig


In [42]:
exectime_end = datetime.datetime.now()

### make (final) pfsConfig and pfsConfigFiber

#### preparation (make pfsConfig)

In [43]:
pfsConfig = erunsim.make_pfsConfig(pfsDesignId, visit, fiberIds, tracts, patches, ras, decs, 
                                   catIds, objIds, targetTypeIds, fiberMags_g, 
                                   pfiNominal_x, pfiNominal_y, pfiCenter_x, pfiCenter_y)

In [44]:
allocNumIter = iteration
allocElapsetime = exectime_end - exectime_start
print('iteration=%d, elapsetime=%.1f (sec.)' % (allocNumIter, allocElapsetime.total_seconds()))

iteration=2, elapsetime=44.1 (sec.)


#### update `pfsConfig`

In [45]:
db.insert_pfsConfig(pfsConfigId=pfsConfigId, pfsDesignId=pfsDesignId, visit0=visit, 
                    raCenter=raCenter, decCenter=decCenter, paConfig=pa, telEl=90.0, insRot=0.0,
                    numSciAllocated=nFiber, numCalAllocated=0, numSkyAllocated=0, numGuideStars=0,
                    exptime=900., minExptime=900., 
                    allocNumIter=allocNumIter, allocElapsetime=allocElapsetime.total_seconds(), allocRmsScatter=0.0, 
                    allocExectime=datetime.datetime.now(), observed=False)

update pfsConfig


#### update `pfsConfigFiber` (may takes a few minitues depending on network environment)

In [46]:
onSources = np.array([True for i in range(nFiber)], dtype='bool')
motorMapSummary = np.array(['moved' for i in range(nFiber)], dtype='U20')
configTime = np.array([allocElapsetime.total_seconds() for i in range(nFiber)], dtype='f4')

db.insert_pfsConfigFiber(pfsConfigFiberId=pfsConfigFiberIds, pfsConfigId=pfsConfigId, fiberId=fiberIds, targetId=targetIds, 
                         tract=tracts, patch=patches, ra=ras, dec=decs, catId=catIds, objId=objIds, targetTypeId=targetTypeIds, 
                         fiberMag_g=fiberMags_g, fiberMag_r=fiberMags_r, fiberMag_i=fiberMags_i, fiberMag_z=fiberMags_z, 
                         fiberMag_y=fiberMags_y, fiberMag_j=fiberMags_j,
                         pfiNominal_x=pfiNominal_x, pfiNominal_y=pfiNominal_y, pfiCenter_x=pfiCenter_x, pfiCenter_y=pfiCenter_y, 
                         pfiDiff_x=pfiDiff_x, pfiDiff_y=pfiDiff_y, mcsCenter_x=mcsCenter_x, mcsCenter_y=mcsCenter_y,
                         motorMapSummary=motorMapSummary, configTime=configTime, onSource=onSources)


update pfsConfigFiber


### close database

In [47]:
db.close()

closed mcs_erun_201908
